# Coloumb Matrix Elements

\begin{align*}
\langle 12| V | 34 \rangle =& 
\delta_{s_1, s_4} \delta_{s_2, s_3} \delta_{m_1 + m_2, m_3 + m_4}
\Big[\prod_{i=1}^4 \frac{n_i!}{(|m_i| + n_i)} \Big]^{1/2}
\sum_{(4)j=0}^n \frac{(-1)^{j_1+j_2+j_3+j_4}}{j_1!j_2!j_3!j_4!} \\
&\times \Big[\prod_{i=1}^n {n_i + |m_i| \choose n_i + j_i} \Big]
\frac{1}{2^{(G+1)/2}} \sum_{(4)l=0}^\gamma (-1)^{\gamma_2 + \gamma_3 - l_2 - l_3} \\
&\times \delta_{l_1 + l_2, l_3 + l_4} \Big[\prod_{i=1}^4 {\gamma_i \choose l_i} \Big] \Gamma\Big(1 + \frac{\Lambda}{2} \Big)\Gamma\Big(\frac{G - \Lambda + 1}{2}\Big)
\end{align*}

The symbols $j_i$ are integer summation indices running from $0$ to $n_i$.
The symbols $\gamma_i$ stand for numbers,
\begin{align*}
\gamma_1 &= j_1 + j_4 + (|m_1| + m_1)/2 + (|m_4| - m_4)/2 \\
\gamma_4 &= j_1 + j_4 + (|m_1| - m_1)/2 + (|m_4| + m_4)/2
\end{align*}
$\gamma_2$ and $\gamma_3$ can be obtained by replacing indices $1 \to 2$ and $4 \to 3$.

Moreover,
\begin{align*}
\sum_{(4)j=0}^n =
\sum_{j_1=0}^{n_1}\sum_{j_2=0}^{n_2}\sum_{j_3=0}^{n_3}\sum_{j_4=0}^{n_4},
\quad 
G = \sum_i \gamma_i, 
\quad
\Lambda = \sum_i l_i
\end{align*}

In [1]:
import numpy as np
from scipy.misc import factorial

In [ ]:
n = np.zeros(4)
m = np.zeros(4)
j = np.zeros(4)

    first_product = 1
    for i in range(4):
        first_product *= (factorial(n[i] / np.abs())